In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import re
import nltk
import gc

## 1. Read train and test files from extraced folder

In [2]:
path='../../Data/Extracted/'
train=pd.read_csv(path+'train.csv').select_dtypes(include='object')
test=pd.read_csv(path+'test.csv').select_dtypes(include='object')

C:\Users\pushk\miniconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
del train['Facility_or_Department'], train['Text_2'], train['Text_3'], train['Text_4'], train['Text_1'], train['SubFund_Description'], train['Position_Extra']
del test['Facility_or_Department'], test['Text_2'], test['Text_3'], test['Text_4'], test['Text_1'], test['SubFund_Description'], test['Position_Extra']
gc.collect()

20

## 2. Clean the text <br>
## 2.1 Fill na by string 'empty'

In [4]:
train.fillna('empty',inplace=True)
test.fillna('empty',inplace=True)

## 2.2 Define function for text preprocessing

In [5]:
def decontraction(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

def preprocess(feat_list):
    for ind,row in enumerate(feat_list):
        # Replace everything except a-z, A-Z and 0-9 by empty string
        row=re.sub(pattern='[^a-zA-Z0-9]+',repl=' ',string=row).strip()
        row=decontraction(row)
        feat_list[ind]=row
    return feat_list

## 2.3 Execute preprocessing

In [6]:
for feat in tqdm(train.columns):
    train[feat]=preprocess(train[feat].str.lower().values)
for feat in tqdm(test.columns):
    test[feat]=preprocess(test[feat].str.lower().values)

## 2.4 Remove stopwords

In [7]:
def remove_stopwords(los):
    '''Takes a list of strings aka 'los' and removes the stop words from each of the strings'''
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize
    
    # Set of stop words
    stop_words = set(stopwords.words('english'))
    return list(map(lambda doc: ' '.join(list(filter(lambda word: word not in stop_words, word_tokenize(doc)))), los))

In [ ]:
for feat in train.columns:
    train[feat]=remove_stopwords(train[feat].values)
for feat in test.columns:
    test[feat]=remove_stopwords(test[feat].values)

In [ ]:
for feat in train.columns:
    for row in range(train.shape[0]):
        if not len(train[feat].iloc[row])>0:
            train[feat].iloc[row]='empty'
            
for feat in test.columns:
    for row in range(test.shape[0]):
        if not len(test[feat].iloc[row])>0:
            test[feat].iloc[row]='empty'

## 2.5 Save data frames to extracted folder

In [ ]:
path='../../Data/Extracted/'

train.to_csv(path+'train_multi_column_text.csv',index=False)
test.to_csv(path+'test_multi_column_text.csv',index=False)